### Reconcile Bank data with Beals data

- Check Beals cumulative payments
    - puts credits/debits into a single column and keeps track of cumulative values - a records with an 'Amount payable' 
      with a non-zero value shows records have been carried forward from previous year or are to be carried forward to
      next.
- Reconcile Bank payments against Beals
    - iterate bank data for each payment from Beals, checking for corresponding data in Beals transactions list.
    - check for Bank records not reconciled to Beals payment
    - check for Beals records with not corresponding Bank payment
    - find Beals payments that include a transaction from previous year (identified by non zero cumulative amount)
    - find Beals transactions that have not been paid for and so are carried forward to next year (reconcile=0 flag)
- Create output spreadsheets
    - included is combo ledger of bank payments received along with Beals transactions in order to check balances

In [1]:
import numpy as np
import pandas as pd

In [2]:
import time
import os.path
import shutil

def backup_file(filename):
    timestr = time.strftime('%Y%m%d-%H%M%S')
    if os.path.isfile(filename):
        shutil.copy2(filename,filename+'.bak_' + timestr)
    return filename

In [3]:
path = 'M:/My Documents/Business/Bugisiw Ltd/TaxReturn/TaxReturn_2020-2021/LLP accounts/'

In [4]:
# Load bank files
#df6=pd.read_csv(path+'6045_s1.csv', parse_dates=['Date'], dayfirst=True)
#df3=pd.read_csv(path+'3072_s1.csv', parse_dates=['Date'], dayfirst=True)
input_file_6045 = path + '6045_categorised.xlsx'
df6=pd.read_excel(input_file_6045,sheet_name='6045')
df6=df6[~df6.Account.isnull()]
input_file_3072 = path + '3072_categorised.xlsx'
df3=pd.read_excel(input_file_3072,sheet_name='3072')
df3=df3[~df3.Account.isnull()]

In [5]:
# Load Beals transactions
#dfB=pd.read_csv(path+'beals.csv', parse_dates=['Date Created'], dayfirst=True)
dfB=pd.read_excel(path+'beals.xlsx', sheet_name='beals', parse_dates=['Date Created'], dayfirst=True)
dfB.insert(0,'Date', dfB['Date Created'].dt.date)
dfB=dfB.sort_values(by=['ID','Date Created'])
dfB['Date'] =  pd.to_datetime(dfB.Date)
#dfB['Item Description'] = dfB.ID + ': ' + dfB['Item Description']   

In [6]:
startdate = pd.to_datetime('2020-04-06').date()
enddate = pd.to_datetime('2021-04-05').date()
dfB=dfB[(dfB.Date >= startdate)&(dfB.Date <= enddate)]
df6=df6[(df6.Date >= startdate)&(df6.Date <= enddate)]
df3=df3[(df3.Date >= startdate)&(df3.Date <= enddate)]

C:\Users\MSTDTP21\.conda\envs\mtpython36\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\MSTDTP21\.conda\envs\mtpython36\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  after removing the cwd from sys.path.
C:\Users\MSTDTP21\.conda\envs\mtpython36\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Comparing Series of datetimes w

### Add cumulative payments to Beals transactions
- it's easier to see missing transacations when a cumulative figure

In [7]:
# Create amount column
for index, row in dfB.iterrows():
    if(row['Credit Amount']>0):
        dfB.at[index,'Amount']=-round(row['Credit Amount'],2)
    elif(row['Debit Amount']>0):
        dfB.at[index,'Amount']=round(row['Debit Amount'],2)

In [8]:
# Track cumulative amounts between payments
cumamount=0.0;
lastID=''
for index, row in dfB.iterrows():
    if(row.ID != lastID):
        cumamount=row.Amount
    else:
        cumamount=cumamount + row.Amount

    dfB.at[index,'CumAmount'] = round(cumamount,2)
    if('Amount payable' in row['Item Description']):
        cumamount=0.0
    lastID=row.ID

### Reconcile Bank payments against Beals

Iterate bank data for each payment from Beals, checking for corresponding data in Beals transactions list.

Mark reconciled data in bank and reconciled Beals payments with 1, mark Beals transactions with account they were paid into.

In [9]:
from datetime import datetime, timedelta

def find_last_rent_received(payment_index, property_code, df_beals):
    rec=df_beals[(df_beals.ID==property_code)&(df_beals['Item Description'].str.contains('Rent for period'))&(df_beals.index<payment_index)]
    if not rec.empty:
        return rec['Item Description'].iloc[-1]
    else:
        return 'NOT FOUND'

def reconcile_bank_with_beals(df_bank, df_beals,bank_label):
    lookback = 5
    errorcount = 0
    for index, row in df_bank[df_bank['Memo'].str.contains("BEALS ESTATE AGENT")].iterrows():
        # Lookback 5 days for payment to appear in Beals statement
        print('Reconciling ' + str(row.Date) + ' ' + row.Memo + ' ' + str(row.Amount))

        startdate = row.Date - timedelta(days=lookback)
        enddate = row.Date
        rec=df_beals.loc[(df_beals['ID']==row.Property)&(df_beals['Debit Amount']==row.Amount)&(df_beals['Date']>=startdate)&(df_beals['Date']<=enddate)&(df_beals.ReconciledBank!=1)]
        if not rec.empty:
            print( str(rec.iloc[0].name)+' With '+ str(rec.Date.iloc[0]) + ' ' + rec['Item Description'].iloc[0] + ' ' + str(rec['Debit Amount'].iloc[0]))
            df_bank.at[index, 'ReconciledBeals']=1
            df_bank.at[index, 'Description']=find_last_rent_received(rec.iloc[0].name,row.Property,df_beals)
            df_beals.at[rec.iloc[0].name, 'ReconciledBank']=1
            df_beals.at[rec.iloc[0].name, 'ReconciledBankAccount']=bank_label
        else:
            print('CANNOT RECONCILE')
            errorcount = errorcount + 1

    print('Errors=' +  str(errorcount))

In [10]:
df6.loc[df6['Memo'].str.contains("BEALS ESTATE AGENT"),'ReconciledBeals']=0
df3.loc[df3['Memo'].str.contains("BEALS ESTATE AGENT"),'ReconciledBeals']=0
dfB['ReconciledBank']=np.nan
dfB['ReconciledBankAccount']=np.nan

#### Reconcile 6045 with Beals

In [11]:
reconcile_bank_with_beals(df6,dfB,6045)

Reconciling 2020-04-08 00:00:00 BEALS ESTATE AGENT    FLAT 1412-14 ALHAM BGC 452.0
497 With 2020-04-06 00:00:00 Amount payable to Mr Mathew Tucker - Flat 14, 12-14 Alhambra Road, Southsea 452.0
Reconciling 2020-04-09 00:00:00 BEALS ESTATE AGENT    FLAT 712-14 ALHAMB BGC 205.7
1403 With 2020-04-07 00:00:00 Amount payable to Mr Mathew Tucker - Flat 7, 12-14 Alhambra Road, Southsea 205.7
Reconciling 2020-04-09 00:00:00 BEALS ESTATE AGENT    FLAT 1169 FAWCETT  BGC 426.55
187 With 2020-04-07 00:00:00 Amount payable to Mr Mathew Tucker - Flat 1, 169 Fawcett Road, Southsea 426.55
Reconciling 2020-04-09 00:00:00 BEALS ESTATE AGENT    196A KINGSTON ROAD BGC 389.3
5 With 2020-04-06 00:00:00 Amount payable to Mr Mathew Tucker - 196a Kingston Road, Portsmouth 389.3
Reconciling 2020-04-09 00:00:00 BEALS ESTATE AGENT    FLAT 3163 FRATTON  BGC 154.17
849 With 2020-04-07 00:00:00 Amount payable to Mr Mathew Tucker - Flat 3, 163 Fratton Road, Portsmouth 154.17
Reconciling 2020-04-09 00:00:00 BEALS ESTA

1262 With 2020-06-03 00:00:00 Amount payable to Mr Mathew Tucker - Flat 6, 16-18 Alhambra Road, Southsea 102.13
Reconciling 2020-06-04 00:00:00 BEALS ESTATE AGENT    FLAT 412-14 ALHAMB BGC 98.97
1018 With 2020-06-03 00:00:00 Amount payable to Mr Mathew Tucker - Flat 4, 12-14 Alhambra Road, Southsea 98.97
Reconciling 2020-06-10 00:00:00 BEALS ESTATE AGENT    FLAT 1412-14 ALHAM BGC 452.0
505 With 2020-06-08 00:00:00 Amount payable to Mr Mathew Tucker - Flat 14, 12-14 Alhambra Road, Southsea 452.0
Reconciling 2020-06-10 00:00:00 BEALS ESTATE AGENT    FLAT 68 ALHAMBRA R BGC 433.94
1333 With 2020-06-08 00:00:00 Amount payable to Mr Mathew Tucker - Flat 6, 8 Alhambra Road, Southsea 433.94
Reconciling 2020-06-10 00:00:00 BEALS ESTATE AGENT    FLAT 412-14 ALHAMB BGC 388.31
1021 With 2020-06-08 00:00:00 Amount payable to Mr Mathew Tucker - Flat 4, 12-14 Alhambra Road, Southsea 388.31
Reconciling 2020-06-15 00:00:00 BEALS ESTATE AGENT    SHOP196 KINGSTON R BGC 314.6
1560 With 2020-06-11 00:00:00

Reconciling 2020-08-18 00:00:00 BEALS ESTATE AGENT    FLAT 3163 FRATTON  BGC 96.89
880 With 2020-08-14 00:00:00 Amount payable to Mr Mathew Tucker - Flat 3, 163 Fratton Road, Portsmouth 96.89
Reconciling 2020-08-25 00:00:00 BEALS ESTATE AGENT    FLAT 1012-14 ALHAM BGC 70.97
87 With 2020-08-21 00:00:00 Amount payable to Mr Mathew Tucker - Flat 10, 12-14 Alhambra Road, Southsea 70.97
Reconciling 2020-08-27 00:00:00 BEALS ESTATE AGENT    FLAT 1321 LONDON R BGC 446.0
465 With 2020-08-25 00:00:00 Amount payable to Mr Mathew Tucker - Flat 1, 321 London Road, Portsmouth 446.0
Reconciling 2020-08-27 00:00:00 BEALS ESTATE AGENT    FLAT 68 ALHAMBRA R BGC 88.56
1350 With 2020-08-25 00:00:00 Amount payable to Mr Mathew Tucker - Flat 6, 8 Alhambra Road, Southsea 88.56
Reconciling 2020-08-27 00:00:00 BEALS ESTATE AGENT    FLAT 1171 FAWCETT  BGC 532.24
288 With 2020-08-25 00:00:00 Amount payable to Mr Mathew Tucker - Flat 1, 171 Fawcett Road, Southsea 532.24
Reconciling 2020-08-27 00:00:00 BEALS ESTA

525 With 2020-11-06 00:00:00 Amount payable to Mr Mathew Tucker - Flat 14, 12-14 Alhambra Road, Southsea 452.0
Reconciling 2020-11-10 00:00:00 BEALS ESTATE AGENT    FLAT 412-14 ALHAMB BGC 444.24
1056 With 2020-11-06 00:00:00 Amount payable to Mr Mathew Tucker - Flat 4, 12-14 Alhambra Road, Southsea 444.24
Reconciling 2020-11-11 00:00:00 BEALS ESTATE AGENT    FLAT 68 ALHAMBRA R BGC 257.7
1368 With 2020-11-09 00:00:00 Amount payable to Mr Mathew Tucker - Flat 6, 8 Alhambra Road, Southsea 257.7
Reconciling 2020-11-12 00:00:00 BEALS ESTATE AGENT    FLAT 2171 FAWCETT  BGC 29.26
739 With 2020-11-10 00:00:00 Amount payable to Mr Mathew Tucker - Flat 2, 171 Fawcett Road, Southsea 29.26
Reconciling 2020-11-13 00:00:00 BEALS ESTATE AGENT    SHOP196 KINGSTON R BGC 314.6
1580 With 2020-11-11 00:00:00 Amount payable to Mr Mathew Tucker - Shop, 196 Kingston Road, Portsmouth 314.6
Reconciling 2020-11-16 00:00:00 BEALS ESTATE AGENT    FLAT 616-18 ALHAMB BGC 132.38
1300 With 2020-11-12 00:00:00 Amount 

308 With 2021-01-26 00:00:00 Amount payable to Mr Mathew Tucker - Flat 1, 171 Fawcett Road, Southsea 538.24
Reconciling 2021-02-01 00:00:00 BEALS ESTATE AGENT    FLAT 1316-18 ALHAM BGC 497.8
438 With 2021-01-28 00:00:00 Amount payable to Mr Mathew Tucker - Flat 13, 16-18 Alhambra Road, Southsea 497.8
Reconciling 2021-02-04 00:00:00 BEALS ESTATE AGENT    196A KINGSTON ROAD BGC 470.32
45 With 2021-02-02 00:00:00 Amount payable to Mr Mathew Tucker - 196a Kingston Road, Portsmouth 470.32
Reconciling 2021-02-04 00:00:00 BEALS ESTATE AGENT    FLAT 3169 FAWCETT  BGC 452.0
966 With 2021-02-02 00:00:00 Amount payable to Mr Mathew Tucker - Flat 3, 169 Fawcett Road, Southsea 452.0
Reconciling 2021-02-10 00:00:00 BEALS ESTATE AGENT    FLAT 68 ALHAMBRA R BGC 151.96
1389 With 2021-02-08 00:00:00 Amount payable to Mr Mathew Tucker - Flat 6, 8 Alhambra Road, Southsea 151.96
Reconciling 2021-02-10 00:00:00 BEALS ESTATE AGENT    FLAT 412-14 ALHAMB BGC 480.79
1076 With 2021-02-08 00:00:00 Amount payable 

1160 With 2021-03-26 00:00:00 Amount payable to Mr Mathew Tucker - Flat 4, 321 London Road, Portsmouth 320.71
Reconciling 2021-03-31 00:00:00 BEALS ESTATE AGENT    FLAT 1316-18 ALHAM BGC 497.8
446 With 2021-03-29 00:00:00 Amount payable to Mr Mathew Tucker - Flat 13, 16-18 Alhambra Road, Southsea 497.8
Errors=0


#### Reconcile 3072 with Beals

In [12]:
reconcile_bank_with_beals(df3,dfB,3072)

Reconciling 2020-04-09 00:00:00 BEALS ESTATE AGENT    FLAT 312-14 ALHAMB BGC 440.29
770 With 2020-04-07 00:00:00 Amount payable to Ms Ivana Valentino - Flat 3, 12-14 Alhambra Road, Southsea 440.29
Reconciling 2020-04-09 00:00:00 BEALS ESTATE AGENT    FLAT 512-14 ALHAMB BGC 203.06
1167 With 2020-04-07 00:00:00 Amount payable to Ms Ivana Valentino - Flat 5, 12-14 Alhambra Road, Southsea 203.06
Reconciling 2020-04-09 00:00:00 BEALS ESTATE AGENT    FLAT 1216-18 ALHAM BGC 138.27
319 With 2020-04-07 00:00:00 Amount payable to Ms Ivana Valentino - Flat 12, 16-18 Alhambra Road, Southsea 138.27
Reconciling 2020-04-09 00:00:00 BEALS ESTATE AGENT    FLAT 1612-14 ALHAM BGC 422.3
547 With 2020-04-07 00:00:00 Amount payable to Ms Ivana Valentino - Flat 16, 12-14 Alhambra Road, Southsea 422.3
Reconciling 2020-04-15 00:00:00 BEALS ESTATE AGENT    FLAT 78 ALHAMBRA R BGC 458.0
1487 With 2020-04-14 00:00:00 Amount payable to Ms Ivana Valentino - Flat 7, 8 Alhambra Road, Southsea 458.0
Reconciling 2020-04

Reconciling 2020-10-22 00:00:00 BEALS ESTATE AGENT    FLAT 312-14 ALHAMB BGC 443.71
811 With 2020-10-20 00:00:00 Amount payable to Ms Ivana Valentino - Flat 3, 12-14 Alhambra Road, Southsea 443.71
Reconciling 2020-10-22 00:00:00 BEALS ESTATE AGENT    FLAT 1612-14 ALHAM BGC 422.29
588 With 2020-10-20 00:00:00 Amount payable to Ms Ivana Valentino - Flat 16, 12-14 Alhambra Road, Southsea 422.29
Reconciling 2020-10-22 00:00:00 BEALS ESTATE AGENT    FLAT 1216-18 ALHAM BGC 396.58
367 With 2020-10-20 00:00:00 Amount payable to Ms Ivana Valentino - Flat 12, 16-18 Alhambra Road, Southsea 396.58
Reconciling 2020-10-27 00:00:00 BEALS ESTATE AGENT    FLAT 512-14 ALHAMB BGC 414.64
1209 With 2020-10-23 00:00:00 Amount payable to Ms Ivana Valentino - Flat 5, 12-14 Alhambra Road, Southsea 414.64
Reconciling 2020-10-30 00:00:00 BEALS ESTATE AGENT    FLAT 1616-18 ALHAM BGC 474.9
655 With 2020-10-28 00:00:00 Amount payable to Ms Ivana Valentino - Flat 16, 16-18 Alhambra Road, Southsea 474.9
Reconciling 2

Reconciling 2021-03-03 00:00:00 BEALS ESTATE AGENT    FLAT 78 ALHAMBRA R BGC 458.0
1545 With 2021-03-01 00:00:00 Amount payable to Ms Ivana Valentino - Flat 7, 8 Alhambra Road, Southsea 458.0
Reconciling 2021-03-11 00:00:00 BEALS ESTATE AGENT    FLAT 1612-14 ALHAM BGC 422.29
620 With 2021-03-09 00:00:00 Amount payable to Ms Ivana Valentino - Flat 16, 12-14 Alhambra Road, Southsea 422.29
Reconciling 2021-03-12 00:00:00 BEALS ESTATE AGENT    FLAT 512-14 ALHAMB BGC 167.94
1241 With 2021-03-10 00:00:00 Amount payable to Ms Ivana Valentino - Flat 5, 12-14 Alhambra Road, Southsea 167.94
Reconciling 2021-03-12 00:00:00 BEALS ESTATE AGENT    FLAT 1216-18 ALHAM BGC 102.16
394 With 2021-03-10 00:00:00 Amount payable to Ms Ivana Valentino - Flat 12, 16-18 Alhambra Road, Southsea 102.16
Reconciling 2021-03-18 00:00:00 BEALS ESTATE AGENT    FLAT 1616-18 ALHAM BGC 2849.4
686 With 2021-03-16 00:00:00 Amount payable to Ms Ivana Valentino - Flat 16, 16-18 Alhambra Road, Southsea 2849.4
Reconciling 2021

#### Tidy reconciliation flag

In [13]:
# Backfill reconciliation flag so we know what bank account all Beals transactions relate to
dfB['ReconciledBank'] = dfB.groupby('ID')['ReconciledBank'].bfill()
dfB['ReconciledBank'].fillna(value=0, inplace=True)
dfB['ReconciledBankAccount'] = dfB.groupby('ID')['ReconciledBankAccount'].bfill()
dfB['ReconciledBankAccount'] = dfB.groupby('ID')['ReconciledBankAccount'].ffill()

#### Error check - Bank transactions not reconciled to Payments

In [14]:
df6.loc[(df6['Memo'].str.contains("BEALS ESTATE AGENT"))&(df6.ReconciledBeals==0)]

,Date,Account,Amount,Subcategory,Memo,Property,Description,Category,ReconciledBeals


In [15]:
df3.loc[(df3['Memo'].str.contains("BEALS ESTATE AGENT"))&(df3.ReconciledBeals==0)]

,Date,Account,Amount,Subcategory,Memo,Property,Description,Category,ReconciledBeals


#### Error check - Payments not reconciled to Bank transaction

In [16]:
dfB.loc[(dfB['Item Description'].str.contains("Amount payable"))&(dfB['Item Description'].str.contains("Tucker"))&(dfB.ReconciledBank==0)]

,Date,ID,Activity I D,Date Created,Item Description,Document Type,Document Number,Debit Amount,Credit Amount,ID Check,Unnamed: 9,Unnamed: 10,Unnamed: 11,Amount,CumAmount,ReconciledBank,ReconciledBankAccount
1164,2021-04-01,F4321LON,799146,2021-04-01 12:17:00,"Amount payable to Mr Mathew Tucker - Flat 4, 3...",Statement,227923,29.71,0.0,PM,NaN,NaN,NaN,29.71,0.0,0.0,6045.0


In [17]:
dfB.loc[(dfB['Item Description'].str.contains("Amount payable"))&(dfB['Item Description'].str.contains("Valentino"))&(dfB.ReconciledBank==0)]

,Date,ID,Activity I D,Date Created,Item Description,Document Type,Document Number,Debit Amount,Credit Amount,ID Check,Unnamed: 9,Unnamed: 10,Unnamed: 11,Amount,CumAmount,ReconciledBank,ReconciledBankAccount


#### Payments with transactions from previous year

The dates on below transactions should all be around start of year. The cumulative amounts of £6 indicate Legislation & Compliance charges in previous period but paid for in this period. Dates later in year can indicate missing data in Beals spreadsheet so check the Beals paper statements match the Beals data.

In [18]:
dfB.loc[(dfB['Item Description'].str.contains("Amount payable"))&(dfB.CumAmount!=0)&(dfB.ReconciledBankAccount==6045)].sort_values(by=['Date'])

,Date,ID,Activity I D,Date Created,Item Description,Document Type,Document Number,Debit Amount,Credit Amount,ID Check,Unnamed: 9,Unnamed: 10,Unnamed: 11,Amount,CumAmount,ReconciledBank,ReconciledBankAccount
497,2020-04-06,F141214ALH,701243,2020-04-06 11:53:55,"Amount payable to Mr Mathew Tucker - Flat 14, ...",Statement,196388,452.00,0.0,PM,NaN,NaN,NaN,452.00,-6.0,1.0,6045.0
56,2020-04-07,F101214ALH,701701,2020-04-07 15:30:20,"Amount payable to Mr Mathew Tucker - Flat 10, ...",Statement,196518,208.48,0.0,PM,NaN,NaN,NaN,208.48,-6.0,1.0,6045.0
187,2020-04-07,F1169FAW,701701,2020-04-07 15:30:20,"Amount payable to Mr Mathew Tucker - Flat 1, 1...",Statement,196511,426.55,0.0,PM,NaN,NaN,NaN,426.55,-6.0,1.0,6045.0
692,2020-04-07,F2171FAW,701701,2020-04-07 15:30:20,"Amount payable to Mr Mathew Tucker - Flat 2, 1...",Statement,196523,422.29,0.0,PM,NaN,NaN,NaN,422.29,-6.0,1.0,6045.0
849,2020-04-07,F3163FRA,701701,2020-04-07 15:30:20,"Amount payable to Mr Mathew Tucker - Flat 3, 1...",Statement,196526,154.17,0.0,PM,NaN,NaN,NaN,154.17,-6.0,1.0,6045.0
1007,2020-04-07,F41214ALH,701701,2020-04-07 15:30:20,"Amount payable to Mr Mathew Tucker - Flat 4, 1...",Statement,196530,433.78,0.0,PM,NaN,NaN,NaN,433.78,-6.0,1.0,6045.0
1084,2020-04-07,F4321LON,701701,2020-04-07 15:30:20,"Amount payable to Mr Mathew Tucker - Flat 4, 3...",Statement,196531,172.77,0.0,PM,NaN,NaN,NaN,172.77,-6.0,1.0,6045.0
1248,2020-04-07,F61618ALH,701701,2020-04-07 15:30:20,"Amount payable to Mr Mathew Tucker - Flat 6, 1...",Statement,196535,31.62,0.0,PM,NaN,NaN,NaN,31.62,-6.0,1.0,6045.0
1403,2020-04-07,F71214ALH,701701,2020-04-07 15:30:20,"Amount payable to Mr Mathew Tucker - Flat 7, 1...",Statement,196537,205.70,0.0,PM,NaN,NaN,NaN,205.70,-6.0,1.0,6045.0
1325,2020-04-08,F68ALH,701860,2020-04-08 11:36:32,"Amount payable to Mr Mathew Tucker - Flat 6, 8...",Statement,196597,452.00,0.0,PM,NaN,NaN,NaN,452.00,-6.0,1.0,6045.0


In [19]:
dfB['CumAmount'].loc[(dfB['Item Description'].str.contains("Amount payable"))&(dfB.CumAmount!=0)&(dfB.ReconciledBankAccount==6045)].sum()

-108.0

In [20]:
# Output to file
#dfB.loc[(dfB['Item Description'].str.contains("Amount payable"))&(dfB.CumAmount!=0)&(dfB.ReconciledBankAccount==6045)].sort_values(by=['Date']).to_excel(path+'6045_prevyr.xlsx',sheet_name='6045', index=False)

In [20]:
dfB.loc[(dfB['Item Description'].str.contains("Amount payable"))&(dfB.CumAmount!=0)&(dfB.ReconciledBankAccount==3072)].sort_values(by=['Date'])

,Date,ID,Activity I D,Date Created,Item Description,Document Type,Document Number,Debit Amount,Credit Amount,ID Check,Unnamed: 9,Unnamed: 10,Unnamed: 11,Amount,CumAmount,ReconciledBank,ReconciledBankAccount
319,2020-04-07,F121618ALH,701701,2020-04-07 15:30:20,Amount payable to Ms Ivana Valentino - Flat 12...,Statement,196519,138.27,0.0,PM,NaN,NaN,NaN,138.27,-6.0,1.0,3072.0
547,2020-04-07,F161214ALH,701701,2020-04-07 15:30:20,Amount payable to Ms Ivana Valentino - Flat 16...,Statement,196522,422.30,0.0,PM,NaN,NaN,NaN,422.30,-6.0,1.0,3072.0
770,2020-04-07,F31214ALH,701701,2020-04-07 15:30:20,"Amount payable to Ms Ivana Valentino - Flat 3,...",Statement,196525,440.29,0.0,PM,NaN,NaN,NaN,440.29,-6.0,1.0,3072.0
1167,2020-04-07,F51214ALH,701701,2020-04-07 15:30:20,"Amount payable to Ms Ivana Valentino - Flat 5,...",Statement,196534,203.06,0.0,PM,NaN,NaN,NaN,203.06,-6.0,1.0,3072.0
1487,2020-04-14,F78ALH,702362,2020-04-14 11:25:33,"Amount payable to Ms Ivana Valentino - Flat 7,...",Statement,196736,458.00,0.0,PM,NaN,NaN,NaN,458.00,-6.0,1.0,3072.0


In [21]:
dfB['CumAmount'].loc[(dfB['Item Description'].str.contains("Amount payable"))&(dfB.CumAmount!=0)&(dfB.ReconciledBankAccount==3072)].sum()

-30.0

In [22]:
# Output to file
#dfB.loc[(dfB['Item Description'].str.contains("Amount payable"))&(dfB.CumAmount!=0)&(dfB.ReconciledBankAccount==3072)].sort_values(by=['Date']).to_excel(path+'3072_prevyr.xlsx',sheet_name='3072', index=False)

#### Payments carried forward to next year

In [22]:
dfB.loc[(dfB.ReconciledBank==0)&(dfB.ReconciledBankAccount==6045)].sort_values(by=['Date'])

,Date,ID,Activity I D,Date Created,Item Description,Document Type,Document Number,Debit Amount,Credit Amount,ID Check,Unnamed: 9,Unnamed: 10,Unnamed: 11,Amount,CumAmount,ReconciledBank,ReconciledBankAccount
53,2021-03-28,196AKIN,797781,2021-03-28 00:33:00,Legislation & Compliance - 196a Kingston Road...,Invoice,295822,6.00,0.00,EX,NaN,NaN,NaN,6.00,6.00,0.0,6045.0
1400,2021-03-28,F68ALH,797781,2021-03-28 00:33:00,"Legislation & Compliance - Flat 6, 8 Alhambra...",Invoice,295955,6.00,0.00,EX,NaN,NaN,NaN,6.00,6.00,0.0,6045.0
1322,2021-03-28,F61618ALH,797781,2021-03-28 00:33:00,"Legislation & Compliance - Flat 6, 16-18 Alha...",Invoice,295785,6.00,0.00,EX,NaN,NaN,NaN,6.00,6.00,0.0,6045.0
1161,2021-03-28,F4321LON,797781,2021-03-28 00:33:00,"Legislation & Compliance - Flat 4, 321 London...",Invoice,295885,6.00,0.00,EX,NaN,NaN,NaN,6.00,6.00,0.0,6045.0
1081,2021-03-28,F41214ALH,797781,2021-03-28 00:33:00,"Legislation & Compliance - Flat 4, 12-14 Alha...",Invoice,295728,6.00,0.00,EX,NaN,NaN,NaN,6.00,6.00,0.0,6045.0
1484,2021-03-28,F71214ALH,797781,2021-03-28 00:33:00,"Legislation & Compliance - Flat 7, 12-14 Alha...",Invoice,295730,6.00,0.00,EX,NaN,NaN,NaN,6.00,6.00,0.0,6045.0
926,2021-03-28,F3163FRA,797781,2021-03-28 00:33:00,"Legislation & Compliance - Flat 3, 163 Fratto...",Invoice,295787,6.00,0.00,EX,NaN,NaN,NaN,6.00,6.00,0.0,6045.0
971,2021-03-28,F3169FAW,797781,2021-03-28 00:33:00,"Legislation & Compliance - Flat 3, 169 Fawcet...",Invoice,295797,6.00,0.00,EX,NaN,NaN,NaN,6.00,6.00,0.0,6045.0
542,2021-03-28,F141214ALH,797781,2021-03-28 00:33:00,"Legislation & Compliance - Flat 14, 12-14 Alh...",Invoice,295722,6.00,0.00,EX,NaN,NaN,NaN,6.00,6.00,0.0,6045.0
494,2021-03-28,F1321LON,797781,2021-03-28 00:33:00,"Legislation & Compliance - Flat 1, 321 London...",Invoice,295882,12.00,0.00,EX,NaN,NaN,NaN,12.00,12.00,0.0,6045.0


In [23]:
dfB['Amount'].loc[(dfB.ReconciledBank==0)&(dfB.ReconciledBankAccount==6045)].sum()

96.0

In [23]:
# Output to file
dfB.loc[(dfB.ReconciledBank==0)&(dfB.ReconciledBankAccount==6045)].sort_values(by=['Date']).to_excel(path+'6045_carriedfwd.xlsx',sheet_name='6045', index=False)

In [24]:
dfB.loc[(dfB.ReconciledBank==0)&(dfB.ReconciledBankAccount==3072)].sort_values(by=['Date'])

,Date,ID,Activity I D,Date Created,Item Description,Document Type,Document Number,Debit Amount,Credit Amount,ID Check,Unnamed: 9,Unnamed: 10,Unnamed: 11,Amount,CumAmount,ReconciledBank,ReconciledBankAccount
398,2021-03-28,F121618ALH,797781,2021-03-28 00:33:00,"Legislation & Compliance - Flat 12, 16-18 Alh...",Invoice,295775,6.0,0.0,EX,NaN,NaN,NaN,6.0,6.0,0.0,3072.0
621,2021-03-28,F161214ALH,797781,2021-03-28 00:33:00,"Legislation & Compliance - Flat 16, 12-14 Alh...",Invoice,295725,6.0,0.0,EX,NaN,NaN,NaN,6.0,6.0,0.0,3072.0
687,2021-03-28,F161618ALH,797781,2021-03-28 00:33:00,"Legislation & Compliance - Flat 16, 16-18 Alh...",Invoice,295777,6.0,0.0,EX,NaN,NaN,NaN,6.0,6.0,0.0,3072.0
846,2021-03-28,F31214ALH,797781,2021-03-28 00:33:00,"Legislation & Compliance - Flat 3, 12-14 Alha...",Invoice,295726,6.0,0.0,EX,NaN,NaN,NaN,6.0,6.0,0.0,3072.0
1245,2021-03-28,F51214ALH,797781,2021-03-28 00:33:00,"Legislation & Compliance - Flat 5, 12-14 Alha...",Invoice,295729,6.0,0.0,EX,NaN,NaN,NaN,6.0,6.0,0.0,3072.0


In [25]:
dfB['Amount'].loc[(dfB.ReconciledBank==0)&(dfB.ReconciledBankAccount==3072)].sum()

30.0

In [24]:
dfB.loc[(dfB.ReconciledBank==0)&(dfB.ReconciledBankAccount==3072)].sort_values(by=['Date']).to_excel(path+'3072_carriedfwd.xlsx',sheet_name='3072', index=False)

### Output files
- save Bank files with Beals payment reconciliation flag
- save Beals files with bank reconciliation flag, bank account it was paid into and cumulative amount

In [26]:
file6=backup_file(path+'6045_categorisedAndBealsReconciled.xlsx')
file3=backup_file(path+'3072_categorisedAndBealsReconciled.xlsx')
fileB=backup_file(path+'beals_bankReconciled.xlsx')
df6.to_excel(file6,sheet_name='6045', index=False)
df3.to_excel(file3,sheet_name='3072', index=False)
dfB.to_excel(fileB,sheet_name='beals', index=False)

### Create Beals/Bank reconciliation check
- this file can be used to check that the bank deposits minus agency fees match the Beals payments

In [ ]:
dfB.rename(columns={'ReconciledBankAccount':'BankAccount'},inplace=True)
dfB.rename(columns={'Item Description':'PostNarrative'},inplace=True)
dfB.rename(columns={'ID':'Property'},inplace=True)

In [ ]:
dfB.loc[(dfB.PostNarrative.str.contains('Rent for period'))&(dfB.BankAccount==6045),'Account']='0001'
dfB.loc[(dfB.PostNarrative.str.contains('Rent for period'))&(dfB.BankAccount==6045),'Account Description']='Rental Income'
dfB.loc[(dfB.PostNarrative.str.contains('Rent for period'))&(dfB.BankAccount==6045),'Category']='RentalIncome'
dfB.loc[(dfB.PostNarrative.str.contains('Rent for period'))&(dfB.BankAccount==3072),'Account']='0002'
dfB.loc[(dfB.PostNarrative.str.contains('Rent for period'))&(dfB.BankAccount==3072),'Account Description']='Rental Income'
dfB.loc[(dfB.PostNarrative.str.contains('Rent for period'))&(dfB.BankAccount==3072),'Category']='RentalIncome'

dfB.loc[(dfB.PostNarrative.str.contains('Legislation & Compliance'))&(dfB.BankAccount==6045),'Account']='1570'
dfB.loc[(dfB.PostNarrative.str.contains('Legislation & Compliance'))&(dfB.BankAccount==6045),'Account Description']='Management Fees'
dfB.loc[(dfB.PostNarrative.str.contains('Legislation & Compliance'))&(dfB.BankAccount==6045),'Category']='AgentFees'
dfB.loc[(dfB.PostNarrative.str.contains('Legislation & Compliance'))&(dfB.BankAccount==3072),'Account']='1570'
dfB.loc[(dfB.PostNarrative.str.contains('Legislation & Compliance'))&(dfB.BankAccount==3072),'Account Description']='Management Fees'
dfB.loc[(dfB.PostNarrative.str.contains('Legislation & Compliance'))&(dfB.BankAccount==3072),'Category']='AgentFees'

dfB.loc[(dfB.PostNarrative.str.contains('Management Fee'))&(dfB.BankAccount==6045),'Account']='1570'
dfB.loc[(dfB.PostNarrative.str.contains('Management Fee'))&(dfB.BankAccount==6045),'Account Description']='Management Fees'
dfB.loc[(dfB.PostNarrative.str.contains('Management Fee'))&(dfB.BankAccount==6045),'Category']='AgentFees'
dfB.loc[(dfB.PostNarrative.str.contains('Management Fee'))&(dfB.BankAccount==3072),'Account']='1570'
dfB.loc[(dfB.PostNarrative.str.contains('Management Fee'))&(dfB.BankAccount==3072),'Account Description']='Management Fees'
dfB.loc[(dfB.PostNarrative.str.contains('Management Fee'))&(dfB.BankAccount==3072),'Category']='AgentFees'

dfB.loc[(dfB.PostNarrative.str.contains('Amount payable'))&(dfB.BankAccount==6045),'Account']='4801'
dfB.loc[(dfB.PostNarrative.str.contains('Amount payable'))&(dfB.BankAccount==6045),'Account Description']='Payment'
dfB.loc[(dfB.PostNarrative.str.contains('Amount payable'))&(dfB.BankAccount==6045),'Category']='Payment'
dfB.loc[(dfB.PostNarrative.str.contains('Amount payable'))&(dfB.BankAccount==3072),'Account']='4801'
dfB.loc[(dfB.PostNarrative.str.contains('Amount payable'))&(dfB.BankAccount==3072),'Account Description']='Payment'
dfB.loc[(dfB.PostNarrative.str.contains('Amount payable'))&(dfB.BankAccount==3072),'Category']='Payment'

dfB.loc[(dfB.PostNarrative.str.contains('Amount payable'))&(dfB.BankAccount==6045),'Account']='4801'
dfB.loc[(dfB.PostNarrative.str.contains('Amount payable'))&(dfB.BankAccount==6045),'Account Description']='Payment'
dfB.loc[(dfB.PostNarrative.str.contains('Amount payable'))&(dfB.BankAccount==6045),'Category']='Payment'
dfB.loc[(dfB.PostNarrative.str.contains('Amount payable'))&(dfB.BankAccount==3072),'Account']='4801'
dfB.loc[(dfB.PostNarrative.str.contains('Amount payable'))&(dfB.BankAccount==3072),'Account Description']='Payment'
dfB.loc[(dfB.PostNarrative.str.contains('Amount payable'))&(dfB.BankAccount==3072),'Category']='Payment'

dfB.loc[(~dfB.PostNarrative.str.contains('Amount payable'))&(~dfB.PostNarrative.str.contains('Rent for period'))&(~dfB.PostNarrative.str.contains('Legislation & Compliance'))&(~dfB.PostNarrative.str.contains('Management Fee'))&(dfB.BankAccount==6045),'Account']='3140'
dfB.loc[(~dfB.PostNarrative.str.contains('Amount payable'))&(~dfB.PostNarrative.str.contains('Rent for period'))&(~dfB.PostNarrative.str.contains('Legislation & Compliance'))&(~dfB.PostNarrative.str.contains('Management Fee'))&(dfB.BankAccount==6045),'Account Description']='Repairs, renewals, maintenance'
dfB.loc[(~dfB.PostNarrative.str.contains('Amount payable'))&(~dfB.PostNarrative.str.contains('Rent for period'))&(~dfB.PostNarrative.str.contains('Legislation & Compliance'))&(~dfB.PostNarrative.str.contains('Management Fee'))&(dfB.BankAccount==6045),'Category']='Maintenance'
dfB.loc[(~dfB.PostNarrative.str.contains('Amount payable'))&(~dfB.PostNarrative.str.contains('Rent for period'))&(~dfB.PostNarrative.str.contains('Legislation & Compliance'))&(~dfB.PostNarrative.str.contains('Management Fee'))&(dfB.BankAccount==3072),'Account']='3140'
dfB.loc[(~dfB.PostNarrative.str.contains('Amount payable'))&(~dfB.PostNarrative.str.contains('Rent for period'))&(~dfB.PostNarrative.str.contains('Legislation & Compliance'))&(~dfB.PostNarrative.str.contains('Management Fee'))&(dfB.BankAccount==3072),'Account Description']='Repairs, renewals, maintenance'
dfB.loc[(~dfB.PostNarrative.str.contains('Amount payable'))&(~dfB.PostNarrative.str.contains('Rent for period'))&(~dfB.PostNarrative.str.contains('Legislation & Compliance'))&(~dfB.PostNarrative.str.contains('Management Fee'))&(dfB.BankAccount==3072),'Category']='Maintenance'


In [ ]:
# Check nothing in Beals remains uncategorised
dfB[dfB.Category.isnull()]

In [ ]:
cols=['BankAccount','Date','Property','PostNarrative','Amount','Account','Account Description','Category']

In [ ]:
dfGr6=dfB.loc[(dfB.PostNarrative.str.contains('Rent for period'))&(dfB.BankAccount==6045)][cols]
dfGr3=dfB.loc[(dfB.PostNarrative.str.contains('Rent for period'))&(dfB.BankAccount==3072)][cols]
dfLc6=dfB.loc[(dfB.PostNarrative.str.contains('Legislation & Compliance'))&(dfB.BankAccount==6045)][cols]
dfLc3=dfB.loc[(dfB.PostNarrative.str.contains('Legislation & Compliance'))&(dfB.BankAccount==3072)][cols]
dfMg6=dfB.loc[(dfB.PostNarrative.str.contains('Management Fee'))&(dfB.BankAccount==6045)][cols]
dfMg3=dfB.loc[(dfB.PostNarrative.str.contains('Management Fee'))&(dfB.BankAccount==3072)][cols]
dfOth6=dfB.loc[(~dfB.PostNarrative.str.contains('Amount payable'))&(~dfB.PostNarrative.str.contains('Rent for period'))&(~dfB.PostNarrative.str.contains('Legislation & Compliance'))&(~dfB.PostNarrative.str.contains('Management Fee'))&(dfB.BankAccount==6045)][cols]
dfOth3=dfB.loc[(~dfB.PostNarrative.str.contains('Amount payable'))&(~dfB.PostNarrative.str.contains('Rent for period'))&(~dfB.PostNarrative.str.contains('Legislation & Compliance'))&(~dfB.PostNarrative.str.contains('Management Fee'))&(dfB.BankAccount==3072)][cols]

In [ ]:
df6.rename(columns={'Account':'BankAccount'},inplace=True)
df6.rename(columns={'Memo':'PostNarrative'},inplace=True)
df6['BankAccount']=6045
df6['Account Description']='Payment'
df6.loc[(df6.Category=='BealsRentalIncome'),'Account']='4801'

df3.rename(columns={'Account':'BankAccount'},inplace=True)
df3.rename(columns={'Memo':'PostNarrative'},inplace=True)
df3['BankAccount']=3072
df3['Account Description']='Payment'
df3.loc[(df6.Category=='BealsRentalIncome'),'Account']='4801'

In [ ]:
dfBk6=df6.loc[(df6.Category=='BealsRentalIncome')][cols]
dfBk3=df3.loc[(df3.Category=='BealsRentalIncome')][cols]

In [ ]:
dfAll6=pd.concat([dfGr6,dfLc6,dfMg6,dfBk6,dfOth6]).sort_values(by=['Property','Date'])
dfAll6.reset_index(drop=True,inplace=True)
dfAll3=pd.concat([dfGr3,dfLc3,dfMg3,dfBk3,dfOth3]).sort_values(by=['Property','Date'])
dfAll3.reset_index(drop=True,inplace=True)

In [ ]:
journals=path + 'journals/'

beals_output_file=backup_file(journals + '3.6.AgentFeesRentalIncomeBankings_Beals_6045.xlsx')
writer = pd.ExcelWriter(beals_output_file, engine='xlsxwriter',datetime_format='dd/mm/yyyy')
dfGr6.to_excel(writer,sheet_name='GrossRent', index=False)
dfLc6.to_excel(writer,sheet_name='Legal', index=False)
dfMg6.to_excel(writer,sheet_name='MgtFee', index=False)
dfOth6.to_excel(writer,sheet_name='Other', index=False)
dfBk6.to_excel(writer,sheet_name='Bank', index=False)
dfAll6.to_excel(writer,sheet_name='All', index=False)
writer.save()

beals_output_file=backup_file(journals + '3.6.AgentFeesRentalIncomeBankings_Beals_3072.xlsx')
writer = pd.ExcelWriter(beals_output_file, engine='xlsxwriter',datetime_format='dd/mm/yyyy')
dfGr3.to_excel(writer,sheet_name='GrossRent', index=False)
dfLc3.to_excel(writer,sheet_name='Legal', index=False)
dfMg3.to_excel(writer,sheet_name='MgtFee', index=False)
dfOth3.to_excel(writer,sheet_name='Other', index=False)
dfBk3.to_excel(writer,sheet_name='Bank', index=False)
dfAll3.to_excel(writer,sheet_name='All', index=False)
writer.save()